# Declaring constants for the notebook


In [7]:
public const string databaseId = "manual-log";
public const string containerId = "aws-entries";

# Initializing the Cosmos client and container



In [8]:
using System;
using Microsoft.Azure.Cosmos;
using System.Collections;
using System.Net;
using Newtonsoft.Json;

// Initialize a new instance of CosmosClient using the built-in account endpoint and key parameters
CosmosClient cosmosClient = new CosmosClient(Cosmos.Endpoint, Cosmos.Key);

Database dataHost = cosmosClient.GetDatabase(databaseId);
Container itemHost = dataHost.GetContainer(containerId);

# Defining the POCO for item


In [9]:
public class LogItem {

    [JsonProperty(PropertyName="id")]
    public string Id {get;set;}

    [JsonProperty(PropertyName="record-name")]
    public string RecordName {get;set;}

    [JsonProperty(PropertyName="tag")]
    public string Tag {get;set;}

    [JsonProperty(PropertyName="start-timestamp")]
    public DateTime? StartedAt {get;set;}

    [JsonProperty(PropertyName="end-timestamp")]
    public DateTime? EndedAt {get;set;}

    [JsonProperty(PropertyName="comment")]
    public string Comment {get;set;}

    [JsonProperty(PropertyName="resource-url")]
    public string ResourceUrl {get;set;}

    [JsonProperty(PropertyName="service")]
    public string ServiceName {get;set;}

    public override string ToString() {
        return $"Log entry is with id- {Id}, name \"{RecordName}\", and started at {StartedAt.ToString()}";
    }
}

# Put data into the structure


In [10]:
LogItem newEntry = new LogItem(){
    Id = "5",
    RecordName = "EC2 - Getting started tutorial",
    Tag = "Aws-Certification",
    StartedAt = DateTime.UtcNow,
    EndedAt = null,
    Comment = "",
    ResourceUrl = "https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/EC2_GetStarted.html",
    ServiceName = "EC2"
};

# Add the item to container


In [2]:
ItemResponse<LogItem> response = await itemHost.CreateItemAsync<LogItem>(newEntry);

Display.AsMarkdown($"Charges of query - **{response.RequestCharge}**");
Display.AsMarkdown($"` {response.Resource.ToString()} `");

Charges of query - **8**

` Log entry is with id- 5, name "EC2 - Getting started tutorial", and started at 1/11/2021 5:37:46 AM `

# Querying back the item


In [9]:
ItemResponse<LogItem> queryResponse = await itemHost.ReadItemAsync<LogItem>(

    partitionKey: new PartitionKey("EC2"),
    id: "5"
);

Display.AsMarkdown($"Retreived document: `{queryResponse.Resource}`");
Display.AsMarkdown($"#### Diagnostics for ReadItemAsync:");
Display.AsMarkdown($"` {queryResponse.Diagnostics.ToString()} `");


LogItem endItem = (LogItem) queryResponse;

endItem.EndedAt = DateTime.UtcNow;

ItemResponse<LogItem> upsertResponse = await itemHost.UpsertItemAsync(partitionKey: new PartitionKey("EC2"), item: endItem);
Display.AsMarkdown($"Charges of query: **{upsertResponse.RequestCharge}**");


Retreived document: `Log entry is with id- 5, name "EC2 - Getting started tutorial", and started at 1/11/2021 5:37:46 AM`

#### Diagnostics for ReadItemAsync:

` {"DiagnosticVersion":"2","Summary":{"StartUtc":"2021-01-11T05:38:50.8449863Z","TotalElapsedTimeInMs":40.3066,"UserAgent":"cosmos-netstandard-sdk/3.12.0|3.11.4|10|X64|Linux 4.19.90-microsoft-standa|.NET Core 3.1.3|","TotalRequestCount":1,"FailedRequestCount":0},"Context":[{"Id":"Microsoft.Azure.Cosmos.Handlers.DiagnosticsHandler","HandlerElapsedTimeInMs":40.2122},{"Id":"SystemInfo","CpuHistory":"(2021-01-11T05:37:32.2961037Z 100.000), (2021-01-11T05:37:42.2971958Z 0.000), (2021-01-11T05:37:52.2935190Z 0.000), (2021-01-11T05:37:52.3065046Z 0.000), (2021-01-11T05:38:02.2976343Z 0.000), (2021-01-11T05:38:42.2987030Z 1.587)"},{"Id":"Microsoft.Azure.Cosmos.Handlers.RetryHandler","HandlerElapsedTimeInMs":40.21},{"Id":"Microsoft.Azure.Cosmos.Handlers.RouterHandler","HandlerElapsedTimeInMs":40.2038},{"Id":"Microsoft.Azure.Cosmos.Handlers.TransportHandler","HandlerElapsedTimeInMs":0.3011},{"Id":"AggregatedClientSideRequestStatistics","ContactedReplicas":[],"RegionsContacted":["https://project-timers-eastus.documents.azure.com:443/"],"FailedReplicas":[]},{"Id":"Microsoft.Azure.Documents.ServerStoreModel","ElapsedTimeInMs":40.1285},{"Id":"StoreResponseStatistics","StartTimeUtc":"2021-01-11T05:38:50.8450933Z","ResponseTimeUtc":"2021-01-11T05:38:50.8851559Z","ElapsedTimeInMs":40.0626,"ResourceType":"Document","OperationType":"Read","LocationEndpoint":"https://project-timers-eastus.documents.azure.com:443/","ActivityId":"2fb103f1-0601-4389-9540-ef2439328f7e","StoreResult":"StorePhysicalAddress: rntbd://cdb-ms-prod-eastus1-fd49.documents.azure.com:14145/apps/8acd1527-a022-4c2a-b2d9-2eea6c3b221d/services/f085975e-f496-48a5-ae43-5ce81ca5fab8/partitions/05064715-9b8e-4a86-8aab-236ca97ea635/replicas/132547803045701812s/, LSN: 6, GlobalCommittedLsn: 5, PartitionKeyRangeId: 0, IsValid: True, StatusCode: 200, SubStatusCode: 0, RequestCharge: 1, ItemLSN: 6, SessionToken: -1#6, UsingLocalLSN: True, TransportException: null"}]} `

Charges of query: **10.86**